# **Большой Бот про всё**
![alt text](big_bot-1.png)

[схематически](https://boards.yandex.ru/whiteboard/?hash=d9431dea9bcda9a491849e7e0af7d37e)

In [1]:
# библиотеки
import pandas as pd
import numpy as np
import os
import sys
import sqlite3
import telebot
from telebot import types
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardRemove
import datetime
import time
from datetime import datetime
import random

# пути
dir_path = os.getcwd().replace("\\", "/")
e_path = "/".join([dir_path, "source", "e"]) + "/"
s_path = "/".join([dir_path, "source", "s"]) + "/"
dm_path = "/".join([dir_path, "source", "dm"]) + "/"
dm_etalon_path = "/".join([dir_path, "source", "dm_etalon"]) + "/"
db_path = "/".join([dir_path, "pb_db.sqlite"])

In [146]:
def transponder():
    """
    Опознание нового юзера: в первой и второй паре цифр пин-кода из большего значение вычесть меньшее, результаты перемножить.
    """
    question = [random.randint(0, 9) for _ in range(4)]
    answer = abs(question[0] - question[1]) * abs(question[2] - question[3])
    return question, answer

def user_db(user_id, date_time, flag):
    """
    Регистрация нового юзера в БД
    """
    global db_path
    query_authorization_reselt = """
    INSERT INTO user_status (user_id, date_time, flag) VALUES (?, ?, ?);
    """
    try:
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        cur.execute(
            query_authorization_reselt,
            (
                user_id,
                date_time,
                flag,
            ),
        )
        conn.commit()
        return True, ""
    except Exception as e:
        return False, e
    
def datastatus(e_path, s_path, dm_path, dm_etalon_path, db_path):
    """
    Загрузка исходников и определение статуса данных, возможности расчётов
    """
    # ошибки
    e_path
    e_filename_lst = os.listdir(e_path)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_error_type = """
    SELECT et.final_state, et.error_type, et.error_type_dhb
    FROM error_type et;
    """
    error_type_query = cur.execute(query_error_type).fetchall()
    df_error_type = pd.DataFrame(error_type_query, columns=["final_state", "error_type", "error_type_dhb"])
    etalon_final_state_list = df_error_type["final_state"].tolist()

    if len(e_filename_lst) != 1:
        e_file_status = """
    📙 ОШИБКИ ЗАПИСИ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        e_filename = e_filename_lst[0]
        e_file_path = "".join([e_path, e_filename])
        e_file_modification = datetime.fromtimestamp(os.path.getmtime(e_file_path)).strftime("%Y-%m-%d %H:%M:%S")

        df_e = pd.read_csv(e_file_path, sep=";")
        df_e = df_e[df_e["Тип записи"] == 3]
        df_e = df_e.merge(df_error_type, on="final_state", how="left")
        df_e["create_ts"] = pd.to_datetime(df_e["create_ts"], format="mixed")
        df_e["create_ts"] = df_e["create_ts"].apply(lambda x: x.replace(tzinfo=None).date())
        e_date_min = df_e["create_ts"].min().strftime("%Y-%m-%d")
        e_date_max = df_e["create_ts"].max().strftime("%Y-%m-%d")

        # критические пропуски данных
        df_nonan = df_e[
            [
                "session_id",
                "Название субъекта РФ",
                "step_id",
                "step_name",
                "error_code",
                "final_state",
                "create_ts",
                "Тип записи",
                "referral_flg",
            ]
        ]
        n = 0
        str_0 = "🔸критические пропуски данных:"
        for column in df_nonan.columns:
            if df_nonan[column].isnull().sum() != 0:
                next_str = f"\n\t- столбец '{column}': {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
                str_0 += next_str
                n += 1
        if n == 0:
            str_0 += " отсутствуют"

        # ошибки данных
        str_1 = "🔸нарушение полноты данных: всего (доля от числа строк на соответствующем этапе):"

        # число строк на соответствующих этапах
        len_ValidatePerson = df_e[df_e["step_name"] == "ValidatePerson"].shape[0]
        len_MO = df_e[df_e["step_name"] == "MO"].shape[0]
        len_ServiceOrSpecs = df_e[df_e["step_name"] == "ServiceOrSpecs"].shape[0]
        len_Slot = df_e[df_e["step_name"] == "Slot"].shape[0]
        len_Book = df_e[df_e["step_name"] == "Book"].shape[0]

        """
        На этапе валидации пациента
        есть данные oid МО
        ИЛИ есть данные о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_ValidatePerson = df_e[
            (df_e["step_name"] == "ValidatePerson")
            & (
                df_e["mo_oid"].notnull()
                | df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе выбора МО
        есть данные oid МО
        ИЛИ есть данные о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_MO = df_e[
            (df_e["step_name"] == "MO")
            & (
                df_e["mo_oid"].notnull()
                | df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе выбора специальности врача
        нет данных об oid МО
        ИЛИ нет данных о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_ServiceOrSpecs = df_e[
            (df_e["step_name"] == "ServiceOrSpecs")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """"
        На этапе выбора слота
        нет данных об oid МО
        ИЛИ нет данных о названии МО
        ИЛИ нет данных о специальтности врача
        ИЛИ есть данные о времени слота
        """
        df_Slot = df_e[
            (df_e["step_name"] == "Slot")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | ~df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе записи
        нет любых данных (ИЛИ)
        """
        df_Book = df_e[
            (df_e["step_name"] == "Book")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | ~df_e["Post_name"].notnull()
                | ~df_e["slot_ts"].notnull()
            )
        ]

        if df_ValidatePerson.shape[0] > 0:
            str_1 += f"\n\t- ValidatePerson: {df_ValidatePerson.shape[0]} ({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        if df_MO.shape[0] > 0:
            str_1 += f"\n\t- MO: {df_MO.shape[0]} ({round(df_MO.shape[0]*100/len_MO, 2)} %)"
        if df_ServiceOrSpecs.shape[0] > 0:
            str_1 += f"\n\t- ServiceOrSpecs: {df_ServiceOrSpecs.shape[0]} ({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        if df_Slot.shape[0] > 0:
            str_1 += f"\n\t- Slot: {df_Slot.shape[0]} ({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        if df_Book.shape[0] > 0:
            str_1 += f"\n\t- Book: {df_Book.shape[0]} ({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        if (
            df_ValidatePerson.shape[0]
            + df_MO.shape[0]
            + df_ServiceOrSpecs.shape[0]
            + df_Slot.shape[0]
            + df_Book.shape[0]
            == 0
        ):
            str_1 += " отсутствуют"

        # нераспознанные final_state
        str_2 = "🔸нераспознанные final_state:"
        final_state_list = df_e["final_state"].tolist()
        unnow_final_state_list = list(
            set([x for x in final_state_list if x not in etalon_final_state_list])
        )
        if len(unnow_final_state_list) > 0:
            for i in range(0, len(unnow_final_state_list)):
                str_2 += f"\n\t- {unnow_final_state_list[i]}"
        else:
            str_2 += " отсутствуют"

        # при подготовке данных в фрейм добавляется одни столбец (error_type) и их становится 14
        e_file_status = f"""
    📙 ОШИБКИ ЗАПИСИ
    🔹файл: {e_filename}
    🔹сохранён: {e_file_modification}
    🔹дата данных: {e_date_max} ({e_date_min})
    🔹число столбцов: {df_e.shape[1] - 2} ({"верное" if df_e.shape[1] == 15 else "неверное"})
    🔹число строк: {df_e.shape[0]}
    {str_0}
    {str_1}
    {str_2}
    """

    # успехи
    s_path
    s_filename_lst = os.listdir(s_path)

    if len(s_filename_lst) != 1:
        s_file_status = """
    📗 УСПЕШНЫЕ СЕССИИ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        s_filename = s_filename_lst[0]
        s_file_path = "".join([s_path, s_filename])
        s_file_modification = datetime.fromtimestamp(os.path.getmtime(s_file_path)).strftime("%Y-%m-%d %H:%M:%S")

        df_s = pd.read_csv(s_file_path, sep=";")
        df_s = df_s[df_s["Тип записи"] == 3]
        df_s["Дата"] = pd.to_datetime(df_s["Дата"], format="mixed")
        df_s["slot_ts"] = pd.to_datetime(df_s["slot_ts"], format="mixed")
        df_s["create_ts"] = pd.to_datetime(df_s["create_ts"], format="mixed")
        df_s["slot_ts"] = df_s["slot_ts"].apply(lambda x: x.replace(tzinfo=None))
        df_s["create_ts"] = df_s["create_ts"].apply(lambda x: x.replace(tzinfo=None))
        s_date_min = df_s["Дата"].min().strftime("%Y-%m-%d")
        s_date_max = df_s["Дата"].max().strftime("%Y-%m-%d")

        # пропуски данных
        missing_s_dict = {}
        for column in df_s.columns:
            missing_s_dict[column] = [
                df_s[column].isnull().sum(),
                round(df_s[column].isnull().sum() * 100 / df_s.shape[0], 2),
            ]
        str_0 = "🔸невалидные пропуски данных: всего (доля от общего числа строк):"
        missing_count = 0
        for k in missing_s_dict.keys():
            if missing_s_dict.get(k)[0] != 0:
                str_0 += (
                    f"\n\t- {k}: {missing_s_dict.get(k)[0]} ({missing_s_dict.get(k)[1]} %)"
                )
                missing_count += missing_s_dict.get(k)[0]
        if missing_count == 0:
            str_0 += " отсутствуют"

        s_file_status = f"""
    📗 УСПЕШНЫЕ СЕССИИ
    🔹файл: {s_filename}
    🔹сохранён: {s_file_modification}
    🔹дата данных: {s_date_max} ({s_date_min})
    🔹число столбцов: {df_s.shape[1]} ({"верное" if df_s.shape[1] == 9 else "неверное"})
    🔹число строк: {df_s.shape[0]}
    {str_0}
    """

    # витрины
    dm_path
    dm_etalon_path
    dm_filename_lst = os.listdir(dm_path)
    dm_etalon_filename_lst = os.listdir(dm_etalon_path)

    if len(dm_filename_lst) != 1:
        dm_file_status = """
    📘 ВИТРИНЫ ДАННЫХ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        dm_filename = dm_filename_lst[0]
        dm_file_path = "".join([dm_path, dm_filename])
        dm_file_modification = datetime.fromtimestamp(os.path.getmtime(dm_file_path)).strftime("%Y-%m-%d %H:%M:%S")

        df_dm = pd.read_excel(
        dm_file_path,
        sheet_name="Sheet1",
        index_col=None,
        )
        
        region_count = df_dm["Регион"].nunique()

        if len(dm_etalon_filename_lst) != 1:
            etalon = "🔹эталонного файла витрин нет или в целевой директории более одного файла"
            df_dm_etalon = pd.DataFrame()
        else:
            dm_etalon_filename = dm_etalon_filename_lst[0]
            dm_etalon_file_path = "".join([dm_etalon_path, dm_etalon_filename])
            dm_etalon_file_modification = datetime.fromtimestamp(os.path.getmtime(dm_etalon_file_path)).strftime("%Y-%m-%d %H:%M:%S")

            df_dm_etalon = pd.read_excel(
            dm_etalon_file_path,
            sheet_name="Sheet1",
            index_col=None,
        )
            etalon_region_count = df_dm_etalon[df_dm_etalon["Статус витрины"] != "Не внедрена"]["Регион"].nunique()

        if df_dm_etalon.shape[0] == 0:
            df_dm
        else:
            df_dm = df_dm.merge(pd.DataFrame(df_dm_etalon[df_dm_etalon["Статус витрины"] != "Не внедрена"]["Регион"].unique()), left_on="Регион", right_on=0, how="inner")
            df_dm = df_dm[[col for col in df_dm.columns[:-1]]]



            etalon = f""" 🔸эталонный файл: {dm_etalon_filename}
     🔸сохранён: {dm_etalon_file_modification}
     🔸количество витринных регионов в эталонном файле: {etalon_region_count}
     🔸количество витринных регионов будет в рассылке: {df_dm["Регион"].nunique()}"""

        df_dm["Период"] = pd.to_datetime(df_dm["Период"], format="%d.%m.%Y")
        dm_date_min = df_dm["Период"].min().strftime("%Y-%m-%d")
        dm_date_max = df_dm["Период"].max().strftime("%Y-%m-%d")

        # проверка на отрицательные значения
        str_0 = "🔸отрицательные значения в валидных столбцах:"
        minus = 0
        for col in df_dm.columns:
            try:
                if df_dm[col].min()<0:
                    str_0 += f"\n\t- {col}"
                    minus +=1
            except Exception:
                pass
        if minus == 0:
            str_0 += " отсутствуют (без учёта стобцов, в которых могут быть нечисловые значения)"

        # проверка на нечисловые значения
        str_1 = "🔸нечисловые значения в валидных столбцах:"
        non_numeric = 0
        for col in df_dm.columns[5:]:
            try:
                if df_dm[col].sum()>=0:
                    pass
            except Exception:
                str_1 += f"\n\t- {col}"
                non_numeric += 1
        if non_numeric == 0:
            str_1 += " отсутствуют"
        
        dm_file_status = f"""
    📘 ВИТРИНЫ ДАННЫХ
    🔹файл: {dm_filename}
    🔹сохранён: {dm_file_modification}
    🔹количество витринных регинов в основном файле: {region_count}
    {etalon}
    🔹дата данных: {dm_date_max} ({dm_date_min})
    🔹число столбцов: {df_dm.shape[1]} ({"верное" if df_dm.shape[1] == 13 else "неверное"})
    🔹число строк: {df_s.shape[0]}
    {str_1}
    {str_0}
    """

        if len(set([e_date_max, e_date_min, s_date_max, s_date_min, dm_date_max, dm_date_min])) > 1:
            fail = True
            result_str_0 = f"""
    🔴Даты данных не совпадают, дальнейший расчёт невозможен!
        """
        else:
            fail = False
            result_str_0 = f"""
    🟢Даты всех наборов данных корректны, совпадают.
        """
        if non_numeric != 0:
            fail = True
            result_str_1 = f"""
    🔴В валидных столбцах файла витрин присутствуют нечисловые значения, дальнейший расчёт невозможен!
        """
        else:
            fail = False
            result_str_1 = f"""
    🟢В валидных столбцах файла витрин нечисловых значений нет.
        """

    status = s_file_status + e_file_status + dm_file_status + result_str_0 + result_str_1

    return df_e, df_s, df_dm, status, fail


def reg_ref(db_path):
    """Возвращает полный справочник регионов (сначала неэталонное написание)"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_regions = """
    SELECT r.reg_name_rus AS reg_name, r.reg_name_cm AS reg_name_data, public_flag
    FROM regions r
    UNION
    SELECT r.reg_name_rus AS reg_name, r.reg_name_mc AS reg_name_data, public_flag
    FROM regions r
    UNION
    SELECT r.reg_name_rus AS reg_name, r.reg_name_pos AS reg_name_data, public_flag
    FROM regions r;
    """
    regions = cur.execute(query_regions).fetchall()
    df_regions = pd.DataFrame(regions, columns=["reg_name", "reg_name_data", "public_flag"])
    return df_regions

def admins(db_path):
    """Возвращает список админов этого вот всего"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_regions = """
    SELECT DISTINCT a.admin_id
    FROM admins a;
    """
    return list(cur.execute(query_regions).fetchall()[0])

def dhb_status(df_e, df_s, db_path):
    """Сверка с дашбордом МЗ РФ"""
    df_reg_ref = reg_ref(db_path)
    #выбор рандомного региона
    random_region_list = df_reg_ref.iloc[random.randint(0, df_reg_ref.shape[0])].tolist()[:2]
    df_nodhb_reg = df_reg_ref[df_reg_ref["public_flag"] == 0]
    nodhb_reg_list = list(set(df_nodhb_reg["reg_name"].tolist() + df_nodhb_reg["reg_name_data"].tolist()))
    df_e_0 = df_e[(df_e["error_type_dhb"].notnull()) & (~df_e["Название субъекта РФ"].isin(nodhb_reg_list))]
    # успехи
    df_s_0 = df_s[~df_s["Название субъекта РФ"].isin(nodhb_reg_list)]
    dhb_status = f"""
    📆 СВЕРКА С ДАШБОРДОМ МИНЗДРАВА РОССИИ
    🇷🇺 Российская Федерация:
    🔹успешных записей: {df_s_0.shape[0]}
    🔹 попыток записи, прерванных пользователем: {df_e_0[df_e_0["error_type_dhb"] == "d"].shape[0]}
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: {df_e_0[df_e_0["error_type_dhb"] == "o"].shape[0]}
    🔸из-за ошибок федерального уровня: {df_e_0[df_e_0["error_type_dhb"] == "r"].shape[0]}
    🔸по техническим причинам в субъектах: {df_e_0[df_e_0["error_type_dhb"] == "t"].shape[0]}
    🏘 Субъект Российской Федерации (определяется случайным выбором):
    🔹регион: {random_region_list[0]}
    🔹успешных записей: {df_s_0[df_s_0["Название субъекта РФ"].isin(random_region_list)].shape[0]}
    🔹 попыток записи, прерванных пользователем: {df_e_0[(df_e_0["error_type_dhb"] == "d")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: {df_e_0[(df_e_0["error_type_dhb"] == "o")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    🔸из-за ошибок федерального уровня: {df_e_0[(df_e_0["error_type_dhb"] == "r")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    🔸по техническим причинам в субъектах: {df_e_0[(df_e_0["error_type_dhb"] == "t")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    """
    return dhb_status

In [56]:
df_e, df_s, df_dm, status, fail = datastatus(e_path, s_path, dm_path, dm_etalon_path, db_path)

In [143]:
print(dhb_status(df_e, df_s, db_path))


    📆 СВЕРКА С ДАШБОРДОМ МИНЗДРАВА РОССИИ
    🇷🇺 Российская Федерация:
    🔹успешных записей: 171710
    🔹 попыток записи, прерванных пользователем: 472269
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: 48122
    🔸из-за ошибок федерального уровня: 30899
    🔸по техническим причинам в субъектах: 36328
    🏘 Субъект Российской Федерации (определяется случайным выбором):
    🔹регион: Республика Адыгея
    🔹успешных записей: 446
    🔹 попыток записи, прерванных пользователем: 1230
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: 21
    🔸из-за ошибок федерального уровня: 89
    🔸по техническим причинам в субъектах: 14
    


In [129]:
df_reg_ref = reg_ref(db_path)
# index_r = random.randint(0, df_reg_ref.shape[0])
random_region_list = df_reg_ref.iloc[random.randint(0, df_reg_ref.shape[0])].tolist()[:2]
random_region_list

['Ленинградская область', 'Ленинградская область']

In [147]:
# бот

TOKEN = "7885371036:AAFtfjid_dRHh2ODPUJ-th-0h8vaxTXbtX0"  # token BigBotForAll@Bot

messages = {}
user_authorization = {}
user_status = {}
dhb_s = {}
fail = {}
# accept = {}
# password_flag = {}
# password_flag_add = {}
# act = {}
# act_reg = {}
# act_password = {}
# region_edit = {}

admin_id = [1236685176] # но это неправильно

bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=["start"])
def start(message):
    global user_id, user_authorization, user_status, fail, dhb_s
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    user_id = message.from_user.id
    user_authorization[user_id] = False
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_user_status = """
    SELECT *
    FROM user_status us
    WHERE us.user_id = ?
    ORDER BY us.date_time DESC
    LIMIT 1;
    """
    user_status_result = cur.execute(query_user_status, (user_id,)).fetchall()

    if not user_status_result:
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Пройти авторизацию", callback_data="authorization")
        )
        bot.send_message(
            message.chat.id,
            f"❔{message.from_user.first_name}, Вы здесь впервые и пока не авторизованы",
            reply_markup=keyboard,
        )
        user_authorization[user_id] = True

    elif user_status_result[0][2] == 0:
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Попробовать ещё раз", callback_data="authorization")
        )
        bot.send_message(
            message.chat.id,
            f"❓{message.from_user.first_name}, ранее Вы уже безуспешно пытались пройти авторизацию.",
            reply_markup=keyboard,
        )
        user_authorization[user_id] = True

    elif user_status_result[0][2] == 1:
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(
            message.chat.id,
            f"""👋
Здравствуйте, {message.from_user.first_name}! Вы авторизованы в системе с {user_status_result[0][1]}.""",
        )
        # загрузка исходных данных и определение статуса
        bot.send_message(
            message.chat.id,
            f"""⏳Определяется статус исходных данных...""",
        )
        sticker = bot.send_sticker(
            message.chat.id,
            "CAACAgIAAxkBAAEM6bpm_mRva4ngeEpx0KVlCuwEUDsYVgACZgADTlzSKbUUiSuZ-b7XNgQ",
        )
        df_e, df_s, df_dm, user_status[user_id], fail[user_id] = datastatus(
            e_path, s_path, dm_path, dm_etalon_path, db_path
        )
        dhb_s[user_id] = dhb_status(df_e, df_s, db_path)
        status = user_status.get(user_id) + dhb_s.get(user_id)
        bot.delete_message(sticker.chat.id, sticker.id)
        bot.send_message(
            message.chat.id,
            f"""📂Статус исходных данных:
{status}""",
        )
        bot.delete_message(message.chat.id, message.message_id + 2)
        if not fail.get(user_id):
            keyboard = InlineKeyboardMarkup()
            keyboard.add(
                InlineKeyboardButton("Выполнить расчёты", callback_data="calculation")
            )
            bot.send_message(
                message.chat.id,
                f"❗️{message.from_user.first_name}, внимательно ознакомьтесь со статусом исходных данных. В них отсутствуют критические ошибки, препятствующие дальнейшей работе, но окончательное решение принимает аналитик.",
                reply_markup=keyboard,
            )
        elif fail.get(user_id):
            bot.send_message(message.chat.id, f"❗️{message.from_user.first_name}, это прискорбно, но состояние исходных данных не позволяет использовать их для расчётов.")
        


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    """авторизация неавторизованных: предлагается пин-код, засим одобрение Админом"""
    global user_id, transponder, question, answer, user_status_result, user_status, fail, accept, user_answer_id, date_time, dhb_s
    admin_id = admins(db_path)
    if call.data == "authorization" and user_authorization.get(call.from_user.id):
        bot.delete_message(call.message.chat.id, call.message.message_id)
        question, answer = transponder()
        user_id = call.from_user.id
        bot.send_message(
            call.message.chat.id,
            "".join([str(x) for x in question]),
        )
    elif call.data == "authorization" and not user_authorization.get(call.from_user.id):
        bot.delete_message(call.message.chat.id, call.message.message_id)
        user_id = call.from_user.id
        bot.send_message(
            call.message.chat.id,
            f"✅{call.from_user.first_name}, Вы были авторизованы в системе {user_status_result[0][1]}.",
        )
    elif call.data == "accept":
        # bot.delete_message(call.message.chat.id, call.message.message_id)
        # user_id = call.from_user.id
        accept = 1
        step_result, e = user_db(user_answer_id, date_time, accept)
        for admin_id in admin_id:
            bot.delete_message(admin_id, call.message.message_id)
        if step_result:
            bot.send_message(
                user_answer_id,
                f"✅{call.from_user.first_name}, одобрение Вашего запроса получено. Для работы воспользуйтесть меню.",
            )
            bot.send_message(
                call.message.chat.id,
                f"✅{call.from_user.first_name}, новый пользователь зарегистрирован.",
            )
        elif not step_result:
            bot.send_message(
                user_answer_id,
                f"⛔️{call.from_user.first_name}, одобрение получено, но при внесении информации в базу данных произошёл сбой: {e}.",
            )
            bot.send_message(
                call.message.chat.id,
                f"⛔️При внесении информации в базу данных произошёл сбой: {e}.",
            )

        user_authorization[user_id] = False
    elif call.data == "reject":
        # bot.delete_message(call.message.chat.id, call.message.message_id)
        # user_id = call.from_user.id
        accept = 0
        step_result, e = user_db(user_answer_id, date_time, accept)
        for admin_id in admin_id:
            bot.delete_message(admin_id, call.message.message_id)
        bot.send_message(
            user_answer_id,
            f"⛔️{call.from_user.first_name}, запрос на авторизацию отколнён.",
        )
        user_authorization[user_id] = False
        

@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global messages, user_authorization, answer, user_db, user_status, fail, user_answer_id, date_time, dhb_s
    admin_id = admins(db_path)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    user_name = message.from_user.username
    user_first_name = message.from_user.first_name
    user_last_name = message.from_user.last_name
    messages[user_id] = message.text
    date_time = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    if user_authorization.get(user_id) and messages.get(user_id) == str(answer):
        # тут запрос админу, можно ли пропускать?
        user_answer_id = user_id
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Принять", callback_data="accept"),
            InlineKeyboardButton("Отклонить", callback_data="reject")
        )
        for admin_id in admin_id:
            bot.send_message(
                admin_id,
                f"Пользователь {user_name} ({user_first_name} {user_last_name}, {user_id}) просит принять его.",
                reply_markup=keyboard,
                )
        bot.send_message(
            message.chat.id,
            f"✅Ответ верный, направлен запрос на авторизацию администратору чат-бота.")
    elif user_authorization.get(user_id) and messages.get(user_id) != str(answer):
        step_result, e = user_db(user_id, date_time, 0)
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(
            message.chat.id,
            f"⛔️Увы, {message.from_user.first_name}, ответ неверный. Для повторной попытки воспользуетесь меню.",
        )
        user_authorization[user_id] = False


# запуск бота
bot.infinity_polling()

2024-10-04 12:23:22,267 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-10-04 12:23:22,269 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [50]:
# подготовка данных, отчёты, заливка в db
# ошибки
e_path
e_filename_lst = os.listdir(e_path)
conn = sqlite3.connect(db_path)
cur = conn.cursor()
query_error_type = """
SELECT et.final_state, et.error_type
FROM error_type et
"""
error_type_query = cur.execute(query_error_type).fetchall()
df_error_type = pd.DataFrame(error_type_query, columns=["final_state", "error_type"])
etalon_final_state_list = df_error_type["final_state"].tolist()

if len(e_filename_lst) != 1:
    e_file_status = """
📙 ОШИБКИ ЗАПИСИ
    🔸в целевой директории нет файла или более одного файла
"""
else:
    e_filename = e_filename_lst[0]
    e_file_path = "".join([e_path, e_filename])
    e_file_modification = datetime.fromtimestamp(os.path.getmtime(e_file_path)).strftime("%Y-%m-%d %H:%M:%S")

    df_e = pd.read_csv(e_file_path, sep=";")
    df_e = df_e[df_e["Тип записи"] == 3]
    df_e = df_e.merge(df_error_type, on="final_state", how="left")
    df_e["create_ts"] = pd.to_datetime(df_e["create_ts"], format="mixed")
    df_e["create_ts"] = df_e["create_ts"].apply(lambda x: x.replace(tzinfo=None).date())
    e_date_min = df_e["create_ts"].min().strftime("%Y-%m-%d")
    e_date_max = df_e["create_ts"].max().strftime("%Y-%m-%d")

    # критические пропуски данных
    df_nonan = df_e[
        [
            "session_id",
            "Название субъекта РФ",
            "step_id",
            "step_name",
            "error_code",
            "final_state",
            "create_ts",
            "Тип записи",
            "referral_flg",
        ]
    ]
    n = 0
    str_0 = "🔸критические пропуски данных:"
    for column in df_nonan.columns:
        if df_nonan[column].isnull().sum() != 0:
            next_str = f"\n\t- столбец '{column}': {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
            str_0 += next_str
            n += 1
    if n == 0:
        str_0 += " отсутствуют"

    # ошибки данных
    str_1 = "🔸нарушение полноты данных: всего (доля от числа строк на соответствующем этапе):"

    # число строк на соответствующих этапах
    len_ValidatePerson = df_e[df_e["step_name"] == "ValidatePerson"].shape[0]
    len_MO = df_e[df_e["step_name"] == "MO"].shape[0]
    len_ServiceOrSpecs = df_e[df_e["step_name"] == "ServiceOrSpecs"].shape[0]
    len_Slot = df_e[df_e["step_name"] == "Slot"].shape[0]
    len_Book = df_e[df_e["step_name"] == "Book"].shape[0]

    """
    На этапе валидации пациента
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ValidatePerson = df_e[
        (df_e["step_name"] == "ValidatePerson")
        & (
            df_e["mo_oid"].notnull()
            | df_e["mo_name"].notnull()
            | df_e["Post_name"].notnull()
            | df_e["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора МО
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_MO = df_e[
        (df_e["step_name"] == "MO")
        & (
            df_e["mo_oid"].notnull()
            | df_e["mo_name"].notnull()
            | df_e["Post_name"].notnull()
            | df_e["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора специальности врача
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ServiceOrSpecs = df_e[
        (df_e["step_name"] == "ServiceOrSpecs")
        & (
            ~df_e["mo_oid"].notnull()
            | ~df_e["mo_name"].notnull()
            | df_e["Post_name"].notnull()
            | df_e["slot_ts"].notnull()
        )
    ]
    """"
    На этапе выбора слота
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ нет данных о специальтности врача
    ИЛИ есть данные о времени слота
    """
    df_Slot = df_e[
        (df_e["step_name"] == "Slot")
        & (
            ~df_e["mo_oid"].notnull()
            | ~df_e["mo_name"].notnull()
            | ~df_e["Post_name"].notnull()
            | df_e["slot_ts"].notnull()
        )
    ]
    """
    На этапе записи
    нет любых данных (ИЛИ)
    """
    df_Book = df_e[
        (df_e["step_name"] == "Book")
        & (
            ~df_e["mo_oid"].notnull()
            | ~df_e["mo_name"].notnull()
            | ~df_e["Post_name"].notnull()
            | ~df_e["slot_ts"].notnull()
        )
    ]

    if df_ValidatePerson.shape[0] > 0:
        str_1 += f"\n\t- ValidatePerson: {df_ValidatePerson.shape[0]} ({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
    if df_MO.shape[0] > 0:
        str_1 += f"\n\t- MO: {df_MO.shape[0]} ({round(df_MO.shape[0]*100/len_MO, 2)} %)"
    if df_ServiceOrSpecs.shape[0] > 0:
        str_1 += f"\n\t- ServiceOrSpecs: {df_ServiceOrSpecs.shape[0]} ({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
    if df_Slot.shape[0] > 0:
        str_1 += f"\n\t- Slot: {df_Slot.shape[0]} ({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
    if df_Book.shape[0] > 0:
        str_1 += f"\n\t- Book: {df_Book.shape[0]} ({round(df_Book.shape[0]*100/len_Book, 2)} %)"
    if (
        df_ValidatePerson.shape[0]
        + df_MO.shape[0]
        + df_ServiceOrSpecs.shape[0]
        + df_Slot.shape[0]
        + df_Book.shape[0]
        == 0
    ):
        str_1 += " отсутствуют"

    # нераспознанные final_state
    str_2 = "🔸нераспознанные final_state:"
    final_state_list = df_e["final_state"].tolist()
    unnow_final_state_list = list(
        set([x for x in final_state_list if x not in etalon_final_state_list])
    )
    if len(unnow_final_state_list) > 0:
        for i in range(0, len(unnow_final_state_list)):
            str_2 += f"\n\t- {unnow_final_state_list[i]}"
    else:
        str_2 += " отсутствуют"

    # при подготовке данных в фрейм добавляется одни столбец (error_type) и их становится 14
    e_file_status = f"""
📙 ОШИБКИ ЗАПИСИ
🔹файл: {e_filename}
🔹сохранён: {e_file_modification}
🔹дата данных: {e_date_max} ({e_date_min})
🔹число столбцов: {df_e.shape[1]} ({"верное" if df_e.shape[1] == 14 else "неверное"})
🔹число строк: {df_e.shape[0]}
{str_0}
{str_1}
{str_2}
"""

# успехи
s_path
s_filename_lst = os.listdir(s_path)

if len(s_filename_lst) != 1:
    s_file_status = """
📗 УСПЕШНЫЕ СЕССИИ
    🔸в целевой директории нет файла или более одного файла
"""
else:
    s_filename = s_filename_lst[0]
    s_file_path = "".join([s_path, s_filename])
    s_file_modification = datetime.fromtimestamp(os.path.getmtime(s_file_path)).strftime("%Y-%m-%d %H:%M:%S")

    df_s = pd.read_csv(s_file_path, sep=";")
    df_s = df_s[df_s["Тип записи"] == 3]
    df_s["Дата"] = pd.to_datetime(df_s["Дата"], format="mixed")
    df_s["slot_ts"] = pd.to_datetime(df_s["slot_ts"], format="mixed")
    df_s["create_ts"] = pd.to_datetime(df_s["create_ts"], format="mixed")
    df_s["slot_ts"] = df_s["slot_ts"].apply(lambda x: x.replace(tzinfo=None))
    df_s["create_ts"] = df_s["create_ts"].apply(lambda x: x.replace(tzinfo=None))
    s_date_min = df_s["Дата"].min().strftime("%Y-%m-%d")
    s_date_max = df_s["Дата"].max().strftime("%Y-%m-%d")

    # пропуски данных
    missing_s_dict = {}
    for column in df_s.columns:
        missing_s_dict[column] = [
            df_s[column].isnull().sum(),
            round(df_s[column].isnull().sum() * 100 / df_s.shape[0], 2),
        ]
    str_0 = "🔸невалидные пропуски данных: всего (доля от общего числа строк):"
    missing_count = 0
    for k in missing_s_dict.keys():
        if missing_s_dict.get(k)[0] != 0:
            str_0 += (
                f"\n\t- {k}: {missing_s_dict.get(k)[0]} ({missing_s_dict.get(k)[1]} %)"
            )
            missing_count += missing_s_dict.get(k)[0]
    if missing_count == 0:
        str_0 += " отсутствуют"

    s_file_status = f"""
📗 УСПЕШНЫЕ СЕССИИ
🔹файл: {s_filename}
🔹сохранён: {s_file_modification}
🔹дата данных: {s_date_max} ({s_date_min})
🔹число столбцов: {df_s.shape[1]} ({"верное" if df_s.shape[1] == 9 else "неверное"})
🔹число строк: {df_s.shape[0]}
{str_0}
"""

# витрины
dm_path
dm_etalon_path
dm_filename_lst = os.listdir(dm_path)
dm_etalon_filename_lst = os.listdir(dm_etalon_path)

if len(dm_filename_lst) != 1:
    dm_file_status = """
📘 ВИТРИНЫ ДАННЫХ
    🔸в целевой директории нет файла или более одного файла
"""
else:
    dm_filename = dm_filename_lst[0]
    dm_file_path = "".join([dm_path, dm_filename])
    dm_file_modification = datetime.fromtimestamp(os.path.getmtime(dm_file_path)).strftime("%Y-%m-%d %H:%M:%S")

    df_dm = pd.read_excel(
    dm_file_path,
    sheet_name="Sheet1",
    index_col=None,
    )
    
    region_count = df_dm["Регион"].nunique()

    if len(dm_etalon_filename_lst) != 1:
        etalon = "🔹эталонного файла витрин нет или в целевой директории более одного файла"
        df_dm_etalon = pd.DataFrame()
    else:
        dm_etalon_filename = dm_etalon_filename_lst[0]
        dm_etalon_file_path = "".join([dm_etalon_path, dm_etalon_filename])
        dm_etalon_file_modification = datetime.fromtimestamp(os.path.getmtime(dm_etalon_file_path)).strftime("%Y-%m-%d %H:%M:%S")

        df_dm_etalon = pd.read_excel(
        dm_etalon_file_path,
        sheet_name="Sheet1",
        index_col=None,
    )
        etalon_region_count = df_dm_etalon[df_dm_etalon["Статус витрины"] != "Не внедрена"]["Регион"].nunique()

    if df_dm_etalon.shape[0] == 0:
        df_dm
    else:
        df_dm = df_dm.merge(pd.DataFrame(df_dm_etalon[df_dm_etalon["Статус витрины"] != "Не внедрена"]["Регион"].unique()), left_on="Регион", right_on=0, how="inner")
        df_dm = df_dm[[col for col in df_dm.columns[:-1]]]



        etalon = f""" ▪️ эталонный файл: {dm_etalon_filename}
 ▪️ сохранён: {dm_etalon_file_modification}
 ▪️ количество витринных регионов в эталонном файле: {etalon_region_count}
 ▪️ количество витринных регионов будет в рассылке: {df_dm["Регион"].nunique()}"""

    df_dm["Период"] = pd.to_datetime(df_dm["Период"], format="%d.%m.%Y")
    dm_date_min = df_dm["Период"].min().strftime("%Y-%m-%d")
    dm_date_max = df_dm["Период"].max().strftime("%Y-%m-%d")

    # проверка на отрицательные значения
    str_0 = "🔸отрицательные значения в валидных столбцах:"
    minus = 0
    for col in df_dm.columns:
        try:
            if df_dm[col].min()<0:
                str_0 += f"\n\t- {col}"
                minus +=1
        except Exception:
            pass
    if minus == 0:
        str_0 += " отсутствуют (без учёта стобцов, в которых могут быть нечисловые значения)"

    # проверка на нечисловые значения
    str_1 = "🔸нечисловые значения в валидных столбцах:"
    non_numeric = 0
    for col in df_dm.columns[5:]:
        try:
            if df_dm[col].sum()>=0:
                pass
        except Exception:
            str_1 += f"\n\t- {col}"
            non_numeric += 1
    if non_numeric == 0:
        str_1 += " отсутствуют"
    
    dm_file_status = f"""
📘 ВИТРИНЫ ДАННЫХ
🔹файл: {dm_filename}
🔹сохранён: {dm_file_modification}
🔹количество витринных регинов в основном файле: {region_count}
{etalon}
🔹дата данных: {dm_date_max} ({dm_date_min})
🔹число столбцов: {df_dm.shape[1]} ({"верное" if df_dm.shape[1] == 13 else "неверное"})
🔹число строк: {df_s.shape[0]}
{str_1}
{str_0}
"""

    if len(set([e_date_max, e_date_min, s_date_max, s_date_min, dm_date_max, dm_date_min])) > 1:
        fail = True
        result_str_0 = f"""
🔴Даты данных не совпадают, дальнейший расчёт невозможен!
    """
    else:
        fail = False
        result_str_0 = f"""
🟢Даты всех наборов данных корректны, совпадают.
    """
    if non_numeric != 0:
        fail = True
        result_str_1 = f"""
🔴В валидных столбцах файла витрин присутствуют нечисловые значения, дальнейший расчёт невозможен!
    """
    else:
        fail = False
        result_str_1 = f"""
🟢В валидных столбцах файла витрин нечисловых значений нет.
    """


status = s_file_status + e_file_status + dm_file_status + result_str_0 + result_str_1

print(status)


📗 УСПЕШНЫЕ СЕССИИ
🔹файл: И38_успешные_сессии_20241002.csv
🔹сохранён: 2024-10-03 08:56:07
🔹дата данных: 2024-10-02 (2024-10-02)
🔹число столбцов: 9 (верное)
🔹число строк: 171875
🔸невалидные пропуски данных: всего (доля от общего числа строк):
	- sp_oid: 2357 (1.37 %)

📙 ОШИБКИ ЗАПИСИ
🔹файл: 02.10.2024.csv
🔹сохранён: 2024-10-03 08:56:45
🔹дата данных: 2024-10-02 (2024-10-02)
🔹число столбцов: 14 (верное)
🔹число строк: 605408
🔸критические пропуски данных:
	- столбец 'Название субъекта РФ': 2 (0.0 %)
🔸нарушение полноты данных: всего (доля от числа строк на соответствующем этапе):
	- ServiceOrSpecs: 3784 (2.86 %)
	- Slot: 3010 (1.83 %)
	- Book: 284 (1.47 %)
🔸нераспознанные final_state: отсутствуют

📘 ВИТРИНЫ ДАННЫХ
🔹файл: dm_2024-10-02.xlsx
🔹сохранён: 2024-10-03 10:10:54
🔹количество витринных регинов в основном файле: 63
 ▪️ эталонный файл: dm_status.xlsx
 ▪️ сохранён: 2024-10-03 10:06:54
 ▪️ количество витринных регионов в эталонном файле: 54
 ▪️ количество витринных регионов будет в рассылк

In [53]:
df_dm

Unnamed: 0  Код                                    Регион     Период   
0             4   30                      Астраханская область 2024-10-02  \
1             5   30                      Астраханская область 2024-10-02   
2             6   30                      Астраханская область 2024-10-02   
3             7   30                      Астраханская область 2024-10-02   
4             8   31                      Белгородская область 2024-10-02   
..          ...  ...                                       ...        ...   
179         223   86  Ханты-Мансийский автономный округ - Югра 2024-10-02   
180         236   89           Ямало-Ненецкий автономный округ 2024-10-02   
181         237   89           Ямало-Ненецкий автономный округ 2024-10-02   
182         238   89           Ямало-Ненецкий автономный округ 2024-10-02   
183         239   89           Ямало-Ненецкий автономный округ 2024-10-02   

                                                   Шаг  Всего  Успешно   
0    Шаг 1. Идентификация пациента в регионе. Уника...   6235     6209  \
1        Шаг 2. Выбор специальности. Уникальные сессии   6209     5719   
2      Шаг 3. Выбор врача и времени. Уникальные сессии   5719     4527   
3                     Шаг 4. Запись. Уникальные сессии   4527     1425   
4    Шаг 1. Идентификация пациента в регионе. Уника...      1        0   
..                                                 ...    ...      ...   
179                   Шаг 4. Запись. Уникальные сессии      0        0   
180  Шаг 1. Идентификация пациента в регионе. Уника...  10588    10525   
181      Шаг 2. Выбор специальности. Уникальные сессии  10525    10146   
182    Шаг 3. Выбор врача и времени. Уникальные сессии  10146     7716   
183                   Шаг 4. Запись. Уникальные сессии   7716     2142   

     ЕПГУ ошибки  Тех. ошибки  Орг. ошибки  ПОДД ошибки  Черновики   
0              0            1           25            0          0  \
1              0          137           11            0        342   
2              0            1          427            0        764   
3              0            0            2          232       2704   
4              0            0            1            0          0   
..           ...          ...          ...          ...        ...   
179            0            0            0            0          0   
180            0            0           63            0          0   
181            0            0           15            0        364   
182            0            0         1368            0       1062   
183            0            6          138          107       5106   

     Бизнес-правила  
0                 0  
1                 0  
2                 0  
3               164  
4                 0  
..              ...  
179               0  
180               0  
181               0  
182               0  
183             217  

[184 rows x 13 columns]

In [67]:
if df_e["create_ts"].min().strftime("%Y-%m-%d") == df_e["create_ts"].max().strftime("%Y-%m-%d"):
    print("равно")

равно


In [68]:
type(df_e["create_ts"].min().strftime("%Y-%m-%d"))

str

In [36]:
query_error_type = """
SELECT et.final_state, et.error_type
FROM error_type et
"""
conn = sqlite3.connect(db_path)
cur = conn.cursor()
error_type_query = cur.execute(query_error_type).fetchall()
df_error_type = pd.DataFrame(error_type_query, columns = ["final_state", "error_type"])

final_state error_type
0   Пациент уже записан на это время к другому спе...          b
1       Пациент уже записан к специалисту в этот день          b
2   Выбранное время доступно только для записи пац...          b
3                                 Значение не найдено          o
4              Ошибка соответствия ответа запросу МИС          t
5                            Сервис МИС приостановлен          t
6                     Услуга посмертно не оказывается          t
7          Внутренняя ошибка РМИС при создании записи          t
8                            Ошибка «РМИС не ответил»          t
9                   Время уже занято другим пациентом          t
10                   Ошибка валидации данных пациента          t
11                             Ошибка валидации схемы          t
12                     Время начала приема уже прошло          t
13          Ошибка доставки формы при записи на прием          t
14                  Истекло время сессии пользователя          t
15  Внутренняя ошибка концентратора при создании з...          r
16   Ошибка во входных параметрах при записи на прием          r
17                              Ошибка «СМЭВ-timeout»          r
18                                  Ошибка отсутсвует          d

In [ ]:
# проверка качества исходников

# загрузка
e_path = "/".join([dir_path, "source", "e"]) + "/"
s_path = "/".join([dir_path, "source", "s"]) + "/"
dm_path = "/".join([dir_path, "source", "dm"]) + "/"
dm_etalon_path = "/".join([dir_path, "source", "dm_etalon"]) + "/"
